In [32]:
# Loading the dataset
import tensorflow as tf

((X_train, y_train), (X_test, y_test)) = tf.keras.datasets.mnist.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0 # normalization

# reshape for image generator
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

print(X_test)

# print(X_train.shape, X_test.shape)
# print(y_train.shape, y_test.shape)

In [ ]:
# Visualizing the dataset
import matplotlib.pyplot as plt

figure = plt.figure()
figure.set_size_inches(15, 5)

axes = []
for i in range(1,6):
    axes.append(figure.add_subplot(1, 5, i))

for i in range(5):
    axes[i].matshow(x_train[i])

In [ ]:
# Building the model
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam

cnn_model = Sequential()

cnn_model.add(Conv2D(10, 3, strides = 1, padding = 'same', activation = 'relu', input_shape = (28, 28, 1)))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Conv2D(10, 3, strides = 1, padding = 'same', activation = 'relu'))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Flatten())

for i in range(3): # 3 hidden layers 
    cnn_model.add(Dense(100))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Activation(activation = 'relu'))

cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(10, activation = 'softmax')) # class: 10, multiclass --> softmax

In [ ]:
# Training the model
cnn_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam(learning_rate=0.001), metrics = ['accuracy'])
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1, verbose=1)

history = cnn_model.fit(X_train, y_train, epochs = 10, batch_size = 50, verbose = 1, validation_data = [X_test, y_test], callbacks=[early_stop])

In [ ]:
# Training the model with image augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                            rotation_range = 50,
                            #width_shift_range = 0.1,
                            #horizontal_flip = True,
                            #vertical_flip = True,
                            #brightness_range=(.9, 1)
                            #zoom_range = 0.7,
                            #height_shift_range=0
                             )

datagen.fit(X_train)

n = 8
fig = plt.figure(figsize = (20,2))

# Creating the image flow with the training dataset
# Loading the certain number of adjusted images
for x_batch in datagen.flow(X_train, batch_size = n):
     for i in range(0,n):
            ax = fig.add_subplot(1, n, i+1)
            ax.imshow(x_batch[i])
     fig.suptitle('Augmented images (rotated 90 degrees)')
     plt.show()
     break;

cnn_model.fit_generator(datagen.flow(X_train, y_train, batch_size = 50), epochs = 10) # fit_generator() when using data made by image generator

score = cnn_model.evaluate(X_test, y_test)
print('Test accuracy', score[1])

In [59]:
# Evaluating the model
evaluation = cnn_model.evaluate(X_test, y_test)

predicted_classes = cnn_model.predict(X_test)
predicted_classes = np.argmax(predicted_classes, axis=1)

L = 7
W = 7
fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(predicted_classes[i], y_test[i]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)

[[ 7  8  9 10]]


In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, predicted_classes)
cm
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True)